In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

In [104]:
# Importamos Librerias
import numpy as np
from collections import Counter
from sortedcontainers import SortedDict

In [14]:
class Datos:

  TiposDeAtributos=('Continuo','Nominal')

  # TODO: procesar el fichero para asignar correctamente las variables tipoAtributos, nombreAtributos, nominalAtributos, datos y diccionarios
  # NOTA: No confundir TiposDeAtributos con tipoAtributos
  def __init__(self, nombreFichero):

      with open(nombreFichero, "r") as f:
        # Guardamos el numero de datos que contiene el DataSet y esta en la primera linea
        self.numDatos = int(f.readline())

        # Guardamos el nombre de los atributos
        self.nombreAtributos = f.readline().strip('\n').split(',')
        #print(self.nombreAtributos)

        # Leemos el tipo de los atributos de las variables y eliminamos el ultimo que es un salto de linea
        self.tipoAtributos = f.readline().strip('\n').split(',')
        #print(self.tipoAtributos)

        # Comprobamos que todos los atributos sean Continuos o Nominales
        if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
            raise ValueError("Tipo de atributo erroneo")

        # Segun el atributo, asignamos True o False.
        self.nominalAtributos = []

        # Guardamos en la lista nominalAtributos en la posicion de cada uno si es o no Nominal
        for tipo in self.tipoAtributos:
            if tipo == self.TiposDeAtributos[0]:
                self.nominalAtributos.append(False)
            else:
                self.nominalAtributos.append(True)
        #print(self.nominalAtributos)

        # Guardamos los datos del fichero y los formateamos, de tal forma que cada linea es una lista
        datos = f.readlines()
        datosFormat = []
        for lista in datos:
            datosFormat.append(lista.strip('\n').split(','))

        # print(set(sorted(datosFormat[0])))
        listaDatosAtributos = []
        for i in range(len(self.tipoAtributos)):
            listaDatosAtributos.append([])

        # Hacemos la traspuesta de los datos que guardamos para que cada lista de atributo guarde todos los datos
        # de cada atributo.
        for lista in datosFormat:
            i = 0
            for item in lista:
                listaDatosAtributos[i].append(item)
                i += 1

        # Ordenamos y hacemos un set para eliminar repetidos.
        i = 0
        for item in listaDatosAtributos:
            listaDatosAtributos[i] = sorted(set(item))
            i += 1


        # Creacion de lista diccionarios, en caso de que el atributo sea Continuo, el diccionario estara vacio
        self.listaDicts = []
        for i in range(len(self.tipoAtributos)):
            self.listaDicts.append({})

        # Creamos el diccionario tal y como se describe en las diapositivas, por orden y asignando valores numericos crecientes
        i = 0
        for atributo in listaDatosAtributos:
            k = 0
            if self.tipoAtributos[i] == "Nominal":
                for dato in atributo:
                    self.listaDicts[i][dato] = k
                    k += 1
            i += 1

        # Creacion de la matriz de datos utilizando el diccionario para mapear los valores
        # En primer lugar, creamos una matriz vacia de tamaña numero de atributos.
        self.datos = np.empty((int(self.numDatos),int(len(self.tipoAtributos))))
        i = 0
        j = 0

        # Metemos los datos en la matriz, mapeando con los diccionarios en el caso de que sean Nominales, y si son continuos normal.
        for i in range(int(self.numDatos)):
            for j in range(len(self.tipoAtributos)):
                if self.tipoAtributos[j] == 'Nominal':
                    self.datos[i][j] = self.listaDicts[j].get(str(datosFormat[i][j]))
                else:
                    self.datos[i][j] = datosFormat[i][j]
        
        print(self.nombreAtributos)
        print(self.listaDicts)
        f.close()


  # TODO: implementar en la practica 1
  def extraeDatos(self, idx):
    return self.datos[idx]


In [15]:
dataset = Datos('lenses.data')

print(dataset.datos)

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]


In [5]:
from abc import ABCMeta,abstractmethod


class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self,train=[],test=[]):
        self.indicesTrain=train
        self.indicesTest=test
    
    def __str__(self):
        return "Train: {}\nTest:  {}".format(str(self.indicesTrain),str(self.indicesTest)) 

#####################################################################################################

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Lista de las particiones
    def __init__(self, nombre=""):
        self.nombreEstrategia = nombre
        self.numeroParticiones = 0
        self.particiones=[]

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self,datos,seed=None):
        pass

In [6]:
class ValidacionSimple(EstrategiaParticionado):
    
    def __init__(self, porcentaje=.70):
        self.porcentaje = porcentaje
        super().__init__("Validacion simple")
        
  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)

        self.numeroParticiones = 1
    
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Creamos la particion, en funcion del porcentaje especificado
        self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos*self.porcentaje)],
                                      indicesAleatorios[int(datos.numDatos*self.porcentaje):])]
        
        return self.particiones
    

In [7]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
print(validacion_simple.particiones[0])

Train: [12 22  8  7  3 16  1 20 23 17 14 18  2 10 15  5  6 21]
Test:  [ 9 13 19  0  4 11]


In [8]:
class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    
    def __init__(self, k):
        self.k = k
        super().__init__("Validacion cruzada")

    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)
        
        self.numeroParticiones = self.k
        
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Hallamos el tamaño de cada bloque
        tamBloque = int(datos.numDatos/self.k)
        
        print(indicesAleatorios)
        
        datosSobran = datos.numDatos - (tamBloque*self.k)
        count = 0
        for i in range(self.k):
            
            train = np.delete(indicesAleatorios, range(i*tamBloque,(i+1)*tamBloque))
            test =  indicesAleatorios[i*tamBloque:(i+1)*tamBloque]
            
            # Caso en el que la cuenta es justa
            if datosSobran == 0:
                self.particiones.append(Particion(train, test))
                
            # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
            if datosSobran > 0:
                count += 1
                particionTest = np.append(test, train[(datos.numDatos - tamBloque)- i - 1])
                particionTrain = np.delete(train, (datos.numDatos - tamBloque)- i - 1)
                datosSobran -= 1
                self.particiones.append(Particion(particionTrain, particionTest))
                
            
           
                

In [9]:
v_cruzada = ValidacionCruzada(5)
v_cruzada.creaParticiones(dataset)

for particion in v_cruzada.particiones:
    print(particion)

[ 6 20 18 22 19 13  7  2 15  8  4  5  0 16  3 17 21 10 11 14 12  9 23  1]
Train: [19 13  7  2 15  8  4  5  0 16  3 17 21 10 11 14 12  9 23]
Test:  [ 6 20 18 22  1]
Train: [ 6 20 18 22 15  8  4  5  0 16  3 17 21 10 11 14 12  9  1]
Test:  [19 13  7  2 23]
Train: [ 6 20 18 22 19 13  7  2  0 16  3 17 21 10 11 14 12 23  1]
Test:  [15  8  4  5  9]
Train: [ 6 20 18 22 19 13  7  2 15  8  4  5 21 10 11 14  9 23  1]
Test:  [ 0 16  3 17 12]
Train: [ 6 20 18 22 19 13  7  2 15  8  4  5  0 16  3 17 12  9 23  1]
Test:  [21 10 11 14]


In [10]:
class Clasificador:
  
    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datos,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
        i = 0
        numDatos = datos.shape()[0]
        error = 0
        for i in range(numDatos):
            if datos[i] != pred[i]:
                error += 1
        err = (error)/(numDatos+0.0)
        return err


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones
    # Validación Simple
        if len(particionado.particiones) == 1:
            clasificador.entrenamiento(dataset.extraeDatos(particionado.particiones[0].indicesTrain), dataset.nominalAtributos, dataset.listaDicts)
            pred = clasificador.clasifica(dataset.extraeDatos(particionado.particiones[0].indicesTest), dataset.nominalAtributos, dataset.listaDicts)
            ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
            if ret > 0:
                return ret
            else:
                return 0
    # Validación Cruzada        
        else:
            for particion in particionado.particiones:
                clasificador.entrenmiento(dataset.extraeDatos(particion.indicesTrain), dataset.nominalAtributos, dataset.listaDicts)
                pred = clasificador.clasifica(dataset.extraeDatos(particion.indicesTest), dataset.nominalAtributos, dataset.listaDicts)
                ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
                errores = np.append(errores,ret)
                #Devolucion de la media de los errores
            return errores

In [117]:
class ClasificadorNaiveBayes(Clasificador):
    
    def entrenamiento(self,datos,datosTrain,atributosDiscretos,diccionario):
        
        # Datos para entrenar el modelo
        self.datosTrain = dataset.extraeDatos(datosTrain)
        
        
        
        # Cargamos todos los datos de la clase del dataset desde la matriz de datos
        self.numClases = datos[:,-1]
    
        # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
        counter = Counter(self.numClases)
        
        # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero 
        # correspondiente a cada clase asignado en el diccionario
        self.dictPrioris={}
        
        for k in counter:
            k = int(k)
            counter[k] = counter[k]/len(self.numClases)
            self.dictPrioris[k] = counter[k]
        
        
        self.dictPrioris = SortedDict(self.dictPrioris)
            
        print(self.dictPrioris)
        
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


In [118]:
clasificador = ClasificadorNaiveBayes()

clasificador.entrenamiento(dataset.datos,validacion_simple.particiones[0].indicesTrain,None,dataset.listaDicts)

SortedDict({0: 0.16666666666666666, 1: 0.20833333333333334, 2: 0.625})
